In [5]:
import http.client
import urllib.parse
from tqdm import tqdm
import json

# Modify path so that we can import local modules into notebook
import os, sys
module_path = os.path.abspath(os.path.join('./data')) 
sys.path.insert(0, module_path)

from data.credentials import ACCESS_KEY

conn = http.client.HTTPConnection('api.mediastack.com')
JSON_FILE = "data.jsonl"
TOTAL = 10000


In [ ]:
pbar = tqdm(total=TOTAL)

data = []
offset = 0

while offset < TOTAL - 100:
    # Fire next request
    params = urllib.parse.urlencode({
        'access_key': ACCESS_KEY,
        'sort': 'published_asc',
        'language': 'en',
        'countries': 'us',
        'limit': 100,
        'offset': offset,
    })
    
    conn.request('GET', '/v1/news?{}'.format(params))

    # Process response
    response = conn.getresponse()
    response = response.read().decode('utf-8')
    response = json.loads(response)

    data.extend(response['data'])
    count = response['pagination']['count']
    offset += count
    pbar.update(count)

    if not count:
        break

pbar.close()

In [4]:
with open(JSON_FILE, "w") as outfile:
    json.dump(data, outfile, indent=True)

print(f"Saved {len(data)} documents to {JSON_FILE}")

Saved 10000 documents to data.jsonl
